In [1]:
from IPython.display import display


# with open('contacts.vcf', 'r') as file:
with open('Antonion and 231 others.vcf', 'r') as file:
    data=file.read().split('BEGIN:VCARD\nVERSION:3.0\n')[1:]

# display(data)

In [2]:
import pandas as pd

csv_example = pd.read_csv('contacts.csv')
result = pd.DataFrame(columns=csv_example.columns, index=range(len(data)))
display(result)

,Name,Given Name,Additional Name,Family Name,Yomi Name,Given Name Yomi,Additional Name Yomi,Family Name Yomi,Name Prefix,Name Suffix,...,Organization 1 - Type,Organization 1 - Name,Organization 1 - Yomi Name,Organization 1 - Title,Organization 1 - Department,Organization 1 - Symbol,Organization 1 - Location,Organization 1 - Job Description,Website 1 - Type,Website 1 - Value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import logging
import sys
logging.basicConfig(level=logging.DEBUG, stream=sys.stderr)

In [4]:
import re

def format_phone(_phone):
    pre=''
    if _phone[:3] == '+86':
        _phone = _phone[3:]
        pre = '+86 '
    elif _phone[:2]=='+1':
        _phone = _phone[2:]
        pre='+1 '

    _phone = re.sub('[^0-9]', '', _phone)
    if _phone[:3] == '010':
        pre+='010 '
        _phone = _phone[3:]
    if len(_phone)==8:
        _phone = "{} {}".format(_phone[:4], _phone[-4:])
    elif len(_phone)==10 or len(_phone)==11:
        _phone = "{} {} {}".format(_phone[:3],_phone[3:-4], _phone[-4:])
    # logging.debug(pre+_phone)
    return pre+_phone

def insert_phone(_i, _type, _value):
    for j in range(1,6):
        if pd.isna(result.loc[_i, 'Phone {} - Value'.format(j)]):
            result.loc[_i, 'Phone {} - Type'.format(j)] = _type
            result.loc[_i, 'Phone {} - Value'.format(j)] = _value
            return

    logging.warning("Some phone number lost! {}".format(entries))
    logging.debug(result.iloc[i, :])

def insert_email(_i, _type, _value):
    for j in range(1,3):
        if pd.isna(result.loc[_i, 'E-mail {} - Value'.format(j)]):
            result.loc[_i, 'E-mail {} - Type'.format(j)] = _type
            result.loc[_i, 'E-mail {} - Value'.format(j)] = _value
            return

    logging.warning("Some Email lost! {}".format(entries))
    logging.debug(result.iloc[i, :])

In [5]:
for i, record in enumerate(data):
    lines=record.split('\n')[:-2]
    for attributes in lines:
        entries = re.split('[;:]', attributes.replace(u'\xa0', ' '))
        key = entries[0]
        # logging.debug(entries)
        if key == 'N':
            if entries[1] != '':
                result.loc[i, 'Family Name'] = entries[1]
            if entries[2] != '':
                result.loc[i, 'Given Name'] = entries[2]
            if entries[3] != '':
                result.loc[i, 'Additional Name'] = entries[3]
                # result.loc[i, 'Nickname'] = entries[3]
        elif key == 'FN':
            result.loc[i, 'Name'] = entries[1]
        elif key == 'X-PHONETIC-FIRST-NAME':
            result.loc[i, 'Given Name Yomi'] = entries[1]
        elif key == 'X-PHONETIC-LAST-NAME':
            result.loc[i, 'Family Name Yomi'] = entries[1]
        elif key == 'NICKNAME':
            result.loc[i, 'Nickname'] = entries[1]
        elif 'TEL' in key:
            phone = format_phone(entries[-1])
            if 'type=WORK' in entries:
                _type='Work'
            elif 'type=HOME' in entries:
                _type='Home'
            else:
                _type='Other'
            insert_phone(i, _type, phone)
        elif key == 'BDAY':
            result.loc[i, 'Birthday'] = entries[-1].replace('1604','-')
        elif key=='ORG':
            result.loc[i, 'Organization 1 - Name'] = entries[1]
            pass
        elif key=='TITLE':
            result.loc[i, 'Organization 1 - Title'] = entries[1]
            pass
        elif key=='NOTE':
            result.loc[i, 'Notes'] = entries[-1]
        elif 'EMAIL' in key:
            email=entries[-1]
            if 'type=WORK' in entries:
                _type='Work'
            elif 'type=HOME' in entries:
                _type='Home'
            else:
                _type='Other'
            insert_email(i, _type, email)
            pass
        elif 'ADR' in key:
            pass
        elif key=='REV':
            pass
        elif key=='PRODID':
            pass
        elif key=='X-SOCIALPROFILE':
            pass
        elif 'Label' in key:
            pass
        elif len(entries) == 1:
            pass
        else:
            # logging.debug(entries)
            # logging.debug(entries)
            pass



display(result.head())

,Name,Given Name,Additional Name,Family Name,Yomi Name,Given Name Yomi,Additional Name Yomi,Family Name Yomi,Name Prefix,Name Suffix,...,Organization 1 - Type,Organization 1 - Name,Organization 1 - Yomi Name,Organization 1 - Title,Organization 1 - Department,Organization 1 - Symbol,Organization 1 - Location,Organization 1 - Job Description,Website 1 - Type,Website 1 - Value
0,Antonion,Antonion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Holly Aun,Holly,NaN,Aun,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Timothy Aun,Timothy,NaN,Aun,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Benjamin,Benjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,毕文珍,文珍,NaN,毕,NaN,Wénzhēn,NaN,Bì,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
result.to_csv('converted_contact.csv', index=None)